This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [ ]:
# First, import the relevant modules
import json
import requests

In [ ]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
URL = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?'
pay_load={
    'start_date':'2017-03-02',
    'end_date':'2017-03-02',
    'api_key':'siQRyjpiRycH87-FCq7v'
}
r = requests.get(URL, params=pay_load)
resp = r.json()

In [ ]:
data = resp['dataset']
print(data.keys())

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


In [ ]:
resp['dataset']

{'id': 10095370,
 'dataset_code': 'AFX_X',
 'database_code': 'FSE',
 'name': 'Carl Zeiss Meditec (AFX_X)',
 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
 'refreshed_at': '2020-12-01T14:48:09.907Z',
 'newest_available_date': '2020-12-01',
 'oldest_available_date': '2000-06-07',
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'frequency': 'daily',
 'type': 'Time Series',
 'premium': False,
 'limit': None,
 'transform': None,
 'column_index': None,
 'start_date': '2017-03-02',
 'end_date': '2017-03-02',
 'data': [['2017-03-02',
   41.38,
   41.39,
   40.76,
   41.17,
   None,
   49863.0,
   2048153.0,
   None,
   None,
   None]],
 'collapse': None,
 'order': None,
 'database_id': 6129}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# Q1
Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [ ]:
def api_call(start_date, end_date):

    pay_load = {
        'start_date':start_date, 
        'end_date':end_date,
        'api_key':API_KEY
    }

    r = requests.get(URL, params=pay_load)

    resp = r.json()

    return resp['dataset']

dataset = api_call(start_date='2017-01-01', end_date='2017-12-31')

# Q2
Convert the returned JSON object into a Python dictionary.

In [ ]:
def convert_json(d):

    daily = d.get('data')
    daily = daily[::-1]
    daily = list(map(list, zip(*daily)))

    columns = data.get('column_names')

    dic = {columns[i]: daily[i] for i in range(len(columns))}

    return dic

stock_info = convert_json(d=dataset)

# Q3
Calculate what the highest and lowest opening prices were for the stock in this period.

In [ ]:
def get_min_max(arr):

    prices = [price for price in arr if price]

    return max(prices), min(prices)

maximum, minimum = get_min_max(arr=stock_info['Open'])
print("Maximum Open:", maximum)
print("Minimum Open:", minimum)

Maximum Open: 53.11
Minimum Open: 34.0


# Q4
What was the largest change in any one day (based on High and Low price)?

In [ ]:
def largest_change(high, low):

    assert len(high) == len(low)
    
    change = 0
    for i in range(len(high)):

        if (high[i] and low[i]) and (high[i] - low[i] > change):

            change = high[i] - low[i]

    return change 

largest_gain = largest_change(high=stock_info['High'], low=stock_info['Low'])
print("Largest 1-Day change", largest_gain) 

Largest 1-Day change 2.8100000000000023


# Q5
What was the largest change between any two days (based on Closing Price)?

In [ ]:
def close_2d(close):

    gain = 0
    for i in range(len(close)-1):
        
        if (close[i] and close[i+1]) and (close[i+1] - close[i] > gain):
            gain = close[i+1] - close[i]
    
    return gain

largest = close_2d(close=stock_info['Close'])
print("Largest change between any two days:", largest)

Largest change between any two days: 1.7199999999999989


# Q6
What was the average daily trading volume during this year?

In [ ]:
def average_annual_volume(volume):

    volume = [vol for vol in volume if vol]
    return sum(volume)/len(volume)

avg_volume = average_annual_volume(volume=stock_info['Traded Volume'])
print("Average daily trading volume:", avg_volume)

Average daily trading volume: 89124.33725490196


# Q7
What was the median trading volume during this year.

In [ ]:
def median_volume(volume):

    volume = [vol for vol in volume if vol]

    volume.sort()

    length = len(volume)

    if length % 2 == 0:
        ind = length/2
        mid = (volume[ind-1] + volume[ind])/2
    else:
        mid = volume[int(length/2 + .5)]

    return mid  

median_volume(volume=stock_info['Traded Volume'])

76600.0